# this notebook is for testing only please use the python file

In [73]:
import pandas as pd
import os 
import numpy as np
FOLDERS = ['../../data/fandom-qa/harry_potter_qa'
           ,'../../data/fandom-qa/witcher_qa_2']

def read_df_from_tsv(file, column_names):
    return pd.read_csv(file, sep='\t', header=None).rename(columns=dict(enumerate(column_names)))


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfIDFCreatorFromArr(arr, min_df=10):
    vectorizer = TfidfVectorizer(strip_accents='unicode', min_df=min_df)
    X = vectorizer.fit_transform(arr)
    return vectorizer, X

vec, X = tfIDFCreatorFromArr(p_df["paragraph"])
X_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())
X_df

,00,000,000094,10,100,1000,101,10th,11,11th,...,zeus,zodiac,zoe,zombie,zonko,zoo,zoomed,zooming,zouwu,zygmunt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
import numpy as np

def getKBestMatchingIndicesForQuestion(question, vectorizer, X, k):
    q_vec = vectorizer.transform([question])
    dot_products = (X@q_vec.T).toarray().flatten()
    return np.argsort(dot_products)[::-1][:k]

# normally the same just in case
def mapToInternalID(df, arr, type='PID'):
    if type == 'PID':
        return df['PID'].iloc[arr].to_numpy()
    elif type == 'QID':
        return df['QID'].iloc[arr].to_numpy()
    
passage_files = [x + '/' + y for x in FOLDERS for y in os.listdir(x) if 'passages' in y and '.tsv' in y]
question_files = [x+ '/' + y for x in FOLDERS for y in os.listdir(x) if 'queries' in y and '.tsv' in y]
triple_files = [x+ '/' + y for x in FOLDERS for y in os.listdir(x) if 'triples' in y and '.tsv' in y]

p_df = read_df_from_tsv(passage_files[0], ['PID', 'paragraph'])
q_df = read_df_from_tsv(question_files[0], ['QID', 'query'])
t_df = read_df_from_tsv(triple_files[0], ['QID', 'PID+', 'PID-'])
vec, X = tfIDFCreatorFromArr(p_df["paragraph"])

question="Who killed Lord Voldemord?"
arr = getKBestMatchingIndicesForQuestion(question, vec, X, 5)
map = mapToInternalID(p_df,arr, type="PID")
pd.options.display.max_colwidth = 300
print(p_df.iloc[map, 1])




3784     [Order of the Phoenix members] Some of the members of the order were killed during the First Wizarding War. These included Marlene McKinnon, who was killed along with her family, Caradoc Dearborn, who was murdered by Death Eaters and his body was never found, Benjy Fenwick, who was killed and mu...
41008    [Aftermath] As the last conflict of the Battle of Hogwarts, this skirmish was a clear victory to the Order of the Phoenix side, all the Death Eaters either killed, defeated or incapacitated. Bellatrix Lestrange was killed in a duel between her and Molly Weasley and Lord Voldemort was defeated an...
34295                                                                                                                                                        [Biography] In 1998, she, along with the other portraits of former Heads, gave Harry Potter a standing ovation after he killed the Dark Lord Voldemort.
42930                                                                    

In [87]:
def isPairInTriples(qid,pid, t_df):
    return t_df.loc[(t_df['QID'] == qid) & (t_df['PID+'] == pid)].any().any()



1.2.2


In [70]:
def printStatistics(result_dfs, k):
    for i, q_df in enumerate(result_dfs):
        hits = len(q_df.loc[q_df['best_k_match'] >= 0])
        total = len(q_df)
        name_dataset = FOLDERS[i].split('/')[-1]
        print(f"In the dataset {name_dataset} tf_idf successfully found the correct passage in the top {k} matches {100.0*hits/total}% of the time")

In [71]:
def getResultDfs(k, FOLDERS):
    passage_files = [x + '/' + y for x in FOLDERS for y in os.listdir(x) if 'passages' in y and '.tsv' in y]
    question_files = [x+ '/' + y for x in FOLDERS for y in os.listdir(x) if 'queries' in y and '.tsv' in y]
    triple_files = [x+ '/' + y for x in FOLDERS for y in os.listdir(x) if 'triples' in y and '.tsv' in y]

    result_dfs = []
    for i in range(len(FOLDERS)):
        p_df = read_df_from_tsv(passage_files[i], ['PID', 'paragraph'])
        q_df = read_df_from_tsv(question_files[i], ['QID', 'query'])
        t_df = read_df_from_tsv(triple_files[i], ['QID', 'PID+', 'PID-'])
        vectorizer, X = tfIDFCreatorFromArr(p_df['paragraph'])
        q_df = q_df.iloc[:100,:]
        q_df["best_k_PID"] = q_df['query'].apply(lambda q: mapToInternalID(p_df, getKBestMatchingIndicesForQuestion(q ,vectorizer, X, k), 'PID'))
        # q_df['best_k_match'] = q_df.apply(lambda x : any([isPairInTriples(x[0], y, t_df) for y in x[2]]), axis=1)
        q_df['best_k_match'] = q_df.apply(lambda x : (lambda arr : -1 if arr[0].size == 0 else arr[0][0]) (np.array([isPairInTriples(x[0], y, t_df) for y in x[2]]).nonzero()) , axis=1)
        result_dfs.append(q_df)
    return result_dfs

printStatistics(getResultDfs(5, FOLDERS), 5)

    

In the dataset harry_potter_qa tf_idf successfully found the correct passage in the top 5 matches 65.0% of the time
In the dataset witcher_qa_2 tf_idf successfully found the correct passage in the top 5 matches 68.0% of the time


In [129]:
print(p_df.iloc[best_ind[:5],1])

3784     [Order of the Phoenix members] Some of the members of the order were killed during the First Wizarding War. These included Marlene McKinnon, who was killed along with her family, Caradoc Dearborn,...
8102                                                                                                [Harry Potter] Harry later avenged his mother when he killed Voldemort at the end of the Battle of Hogwarts.
41007                              [The skirmish] When Bellatrix was killed, Voldemort turned on Mrs Weasley but Harry Potter revealed himself starting the last duel of the War, which was won by Harry Potter.
41008    [Aftermath] As the last conflict of the Battle of Hogwarts, this skirmish was a clear victory to the Order of the Phoenix side, all the Death Eaters either killed, defeated or incapacitated. Bella...
30968    [Biography] This Death Eater shot out a Killing Curse that killed Harry Potter's pet owl, Hedwig. It is likely the curse that killed Hedwig was intended fo

In [125]:
pd.options.display.max_colwidth = 200

In [ ]:
p_df = pd.read_csv('../../data/fandom-qa/harry_potter_qa/passages.train.tsv', sep='\t', header=None)
p_df = p_df.rename(columns={0: 'pid', 1: 'paragraph'})
p_df